In [ ]:
import os
import nltk
from nltk.tokenize import RegexpTokenizer
import numpy as np
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-chinese')

In [ ]:
model= BertModel.from_pretrained("bert-base-chinese")

In [ ]:
def cos_sim(vector_a, vector_b):
        vector_a = np.mat(vector_a)
        vector_b = np.mat(vector_b)
        num = float(vector_a * vector_b.T)
        denom = np.linalg.norm(vector_a) * np.linalg.norm(vector_b)
        sim = num / denom
        print("cos",sim)
        return sim
        
def cac_bert_sim(refs, cand):
    refs = ["".join(ref) for ref in refs]
    cand = "".join(cand).lower()
    print(refs)
    print(cand)
    sims = 0
    for i in range(len(refs)):
        ref = refs[i]
        encoded_input_ref = tokenizer_bert(ref, return_tensors='pt')
        encoded_input_cand = tokenizer_bert(cand, return_tensors='pt')
        output_ref = model(**encoded_input_ref)[1][0].detach().numpy()
        output_cand = model(**encoded_input_cand)[1][0].detach().numpy()
        sims+=cos_sim(output_ref, output_cand)
    print(sims/len(refs))
    return sims/len(refs)

In [ ]:
def cac_avg(l):
    s = 0
    for i in l:
        s+=float(i)
    return s/len(l)
    
def cal_bert(references, candidate):
    references = ["".join(tokenizer.tokenize(i)) for i in references]
    candidate = "".join(tokenizer.tokenize(candidate))
    refs = []
    for ref in references:
        t = []
        sents  = nltk.sent_tokenize(ref)
        words = []
        for sent in sents:
            words.extend(nltk.word_tokenize(sent))
        refs.append(words)
    sents = nltk.sent_tokenize(candidate)
    cands = []
    for sent in sents:
        cands.extend(nltk.word_tokenize(sent))
    #import pdb;pdb.set_trace()
    refs=[ref[0] for ref in refs]
    cands = cands[0]
    score = cac_bert_sim(refs, cands)
    return score
import spacy
nlp =spacy.load('en_core_web_sm')
def cut(s):
    doc = nlp(s)
    tokens = [i.text for i in doc]
    return " ".join(tokens)

In [ ]:
refs = []
with open("./references/EC/1.EC ref.txt", encoding="utf8") as f:
    refs.append(f.readlines())
with open("./references/EC/2.EC ref.txt", encoding="utf8") as f:
    refs.append(f.readlines())
with open("./references/EC/3.EC ref.txt", encoding="utf8") as f:
    refs.append(f.readlines())
with open("./references/EC/4.EC ref.txt", encoding="utf8") as f:
    refs.append(f.readlines())

p_refs = []
n=0
for ref in refs:
    t = []
    n+=1
    for s in ref:
        if len(s)>5:
            a=s[s.index(".")+1:].strip().lower()
            t.append(a)
    p_refs.append(t)
refs = p_refs
   
file_path = "./E-C transcripts-sentence-txt/"

files = []
for i,j,k in os.walk(file_path):
    files.extend(k)


In [ ]:
bert_dict = {}
for fn in files:
    with open(file_path+fn, "r", encoding="utf8") as f:
        pgs = f.readlines()
        t=[]
        for s in pgs:
            if len(s)>5:
                a=s[s.index(".")+1:].strip().lower()
                t.append(a)
        pgs = t
        key = fn.split(".")[0]
        bert_dict[key] = {"bert_val":[],"avg_bert":0}           
        for i in range(len(pgs)):
            p_refs =[] 
            for j in range(len(refs)):         
                p_refs.append(refs[j][i])
            bert_scores = cal_bert(p_refs, pgs[i])
            bert_dict[key]["bert_val"].append(bert_scores)
        bert_dict[key]["avg_bert"] = cac_avg(bert_dict[key]["bert_val"])
import json
with open("ec_bert_score.json", "w", encoding="utf8") as f:
    json.dump(bert_dict,f)

In [ ]:
with open("ec_bert_score.json", encoding="utf8") as f:
    r = json.load(f)

In [ ]:
for k in r:
    print(str(r[k]["avg_bert"]))